[Home](../../README.md)

### Data Wrangling

This is a demonstration of data wrangling using [Pandas](https://pandas.pydata.org/) the library for data analysis and manipulation.

This Jupyter Notepad demonstrates different processes you can apply to your data to prepare it for feature engineering and model training. For this demonstration we will wrangle the diabetes data set you previewed in the last Jupyter Notebook.

> [!Note]
> None of these processes are destructive to the source CSV as long as you save the modified data to a new CSV.

#### Load the required dependencies

In [3]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

####  Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [4]:
data_frame = pd.read_csv("2.1.2.Diabeties_Sample_Data.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

In [5]:
data_frame.isnull().sum()

DoB       0
DoT       0
SEX       1
BMI       0
BP        0
TC        0
BGU       0
FDR       0
Target    1
dtype: int64

If you have null data there are many ways to deal with the empty/null values. These are the two most common approaches.
1. Remove any row with a null value with a `dropna()` method call.
2. Replace missing values with another value with a `fillna()` method call. Generally, we use mean value for numerical columns because it may cause minimal changes in your mathematical analysis while maintaining the original size of the data.

Students should reflect why this example removes the null 'SEX' but replacing the mean 'Target'?

In [6]:
# Remove Null values
data_frame = data_frame.dropna(subset=['SEX'])
data_frame.isnull().sum()

DoB       0
DoT       0
SEX       0
BMI       0
BP        0
TC        0
BGU       0
FDR       0
Target    1
dtype: int64

In [7]:
# Replace Null values with the mode and mean value for the column
data_frame['SEX'] = data_frame['SEX'].fillna(data_frame['SEX'].mode()[0])
data_frame['Target'] = data_frame['Target'].fillna(data_frame['Target'].mean())

#### Remove Duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

In [8]:
data_frame.duplicated().sum()

np.int64(5)

The `drop_duplicates()` method call can be then stored back onto the data_frame variable removing the duplicates.

In [9]:
# Remove duplicates
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

np.int64(0)

#### Replace data

We can run a lambda function on a column to modify its values. For a simple example, let’s convert the Sex to lowercase. To run a function over a complete column, we can use the apply method which iterates over each row and modifies the values.

In [10]:
# Replace data
data_frame['SEX'] = data_frame['SEX'].apply(lambda x: x.lower())
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: 'male' if gender.lower() == 'male' else 'female')

We can check that there are no data entry errors by the `unique()` method call.

In [11]:
data_frame['SEX'].unique()

array(['female', 'male'], dtype=object)

In [12]:
data_frame['SEX'] = data_frame['SEX'].apply(lambda gender: 'male' if gender.lower() == 'male' else 'female')
data_frame['SEX'].unique()

array(['female', 'male'], dtype=object)

#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

In [15]:
# Remove outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]
    return df

data_frame = remove_outliers(data_frame, 'BP')
data_frame = remove_outliers(data_frame, 'BMI')
data_frame = remove_outliers(data_frame, 'BGU')


#### Scaling features to a common range

Scaling the features makes it easier for machine learning algorithms to find the optimal solution, as the different scales of the features do not influence them.

In [16]:
# Scale features
scaler = MinMaxScaler()
data_frame[['BP', 'BMI', 'BGU']] = scaler.fit_transform(data_frame[['BP', 'BMI', 'BGU']])

> [!important]
> You need to save the calculations for each dataset you scale for scaling new values for prediction.

scale BMI

In [17]:
scale_feature = 'BMI'

#the minimum value with space for outliers
MIN_BMI = 12

#the maximum value with space for outliers
MAX_BMI = 46

#scale features
data_frame[scale_feature] = [(X - MIN_BMI) / (MAX_BMI - MIN_BMI) for X in data_frame[scale_feature]]

data_frame.describe()

,BMI,BP,TC,BGU,FDR,Target
count,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000
mean,-0.340943,0.456530,4.037896,0.487971,1.070922,148.946369
std,0.006146,0.190656,1.279464,0.205929,0.828795,75.376514
min,-0.352941,0.000000,2.000000,0.000000,0.000000,25.000000
25%,-0.345441,0.309859,3.000000,0.343137,0.000000,85.000000
50%,-0.341765,0.436620,4.000000,0.490196,1.000000,138.000000
75%,-0.336765,0.598592,5.000000,0.627451,2.000000,202.000000
max,-0.323529,1.000000,9.090000,1.000000,3.000000,341.000000


scale BGU

In [18]:
scale_feature = 'BGU'

#the minimum value with space for outliers
MIN_BGU = 50

#the maximum value with space for outliers
MAX_BGU = 132

#scale features
data_frame[scale_feature] = [(X - MIN_BGU) / (MAX_BGU - MIN_BGU) for X in data_frame[scale_feature]]

data_frame.describe()

,BMI,BP,TC,BGU,FDR,Target
count,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000
mean,-0.340943,0.456530,4.037896,-0.603805,1.070922,148.946369
std,0.006146,0.190656,1.279464,0.002511,0.828795,75.376514
min,-0.352941,0.000000,2.000000,-0.609756,0.000000,25.000000
25%,-0.345441,0.309859,3.000000,-0.605571,0.000000,85.000000
50%,-0.341765,0.436620,4.000000,-0.603778,1.000000,138.000000
75%,-0.336765,0.598592,5.000000,-0.602104,2.000000,202.000000
max,-0.323529,1.000000,9.090000,-0.597561,3.000000,341.000000


#### Save the wrangled data to CSV

In [19]:
data_frame.to_csv('../2.2.Feature_Engineering/2.2.1.wrangled_data.csv', index=False)